In [1]:
from sklearn import *
import pandas as pd


In [2]:
# Read textfiles into dataframes
df_smoker = pd.read_csv('./data/internal_smokerslung.expression.txt', sep="\t", index_col=0).transpose()
df_smoker_meta = pd.read_csv('./data/internal_smokerslung.meta.txt', sep="\t", index_col=0)
df_non_smoker = pd.read_csv('./data/internal_nonsmokerslung.expression.txt', sep="\t", index_col=0).transpose()
df_non_smoker_meta = pd.read_csv('./data/internal_nonsmokerslung.meta.txt', sep="\t", index_col=0)

print ("DF SMOKER");
print(df_smoker);
print("DF SMOKER META");
print(df_smoker_meta);
print("DF NON-SMOKER");
print(df_non_smoker);
print("DF NON-SMOKER META");
print(df_non_smoker_meta);

DF SMOKER
                     ACE2  TMPRSS2  IL6R  IL6ST  PCSK1   IL6  PCSK2  CTSE  \
1B_neg_cell111_HU48   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
1B_neg_cell292_HU48   0.0      0.0   0.0    4.0    0.0  14.0    0.0   0.0   
1B_neg_cell448_HU48   0.0      0.0   0.0    0.0    0.0  20.0    0.0   0.0   
1B_neg_cell449_HU48   0.0      0.0   1.0    0.0    0.0   1.0    0.0   0.0   
1B_neg_cell503_HU48   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
...                   ...      ...   ...    ...    ...   ...    ...   ...   
RBI_cell735720_HU40   0.0      0.0   0.0    3.0    0.0   0.0    0.0   0.0   
RBI_cell736255_HU40   0.0      1.0   0.0    3.0    0.0   0.0    0.0   0.0   
RBI_cell736309_HU40   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
RBI_cell736406_HU40   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
RBI_cell736746_HU40   0.0      0.0   1.0    1.0    0.0   0.0    0.0   0.0   

                     MYRF  MAG  ...  PCSK7  C1R   C2   C3   C5  C

In [3]:
# countnumber of smoker and non-smoker samples
n_smoker = len(df_smoker)
n_non_smoker = len(df_non_smoker)
print(n_smoker);
print(n_non_smoker);
# resample the non-smoker data to have the same number of samples as the smoker data
df_non_smoker_resampled = df_non_smoker.sample(n=n_smoker, replace=False, random_state=42);
# label the data as smoke=yes or smoke=no in both dataframes
df_smoker['smoker'] = 'yes';
df_non_smoker_resampled['smoker'] = 'no';
# combine the data frames
df_combo_balanced = pd.concat([df_smoker, df_non_smoker_resampled], axis=0, ignore_index=True);
print(df_smoker);

96786
277224
                     ACE2  TMPRSS2  IL6R  IL6ST  PCSK1   IL6  PCSK2  CTSE  \
1B_neg_cell111_HU48   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
1B_neg_cell292_HU48   0.0      0.0   0.0    4.0    0.0  14.0    0.0   0.0   
1B_neg_cell448_HU48   0.0      0.0   0.0    0.0    0.0  20.0    0.0   0.0   
1B_neg_cell449_HU48   0.0      0.0   1.0    0.0    0.0   1.0    0.0   0.0   
1B_neg_cell503_HU48   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
...                   ...      ...   ...    ...    ...   ...    ...   ...   
RBI_cell735720_HU40   0.0      0.0   0.0    3.0    0.0   0.0    0.0   0.0   
RBI_cell736255_HU40   0.0      1.0   0.0    3.0    0.0   0.0    0.0   0.0   
RBI_cell736309_HU40   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
RBI_cell736406_HU40   0.0      0.0   0.0    0.0    0.0   0.0    0.0   0.0   
RBI_cell736746_HU40   0.0      0.0   1.0    1.0    0.0   0.0    0.0   0.0   

                     MYRF  MAG  ...  C1R   C2   C3   C5  CFI  

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

X = df_combo_balanced.drop(['smoker'], axis=1)
y = df_combo_balanced['smoker']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
y_pred = dummy_clf.predict(X_test)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.49870849979335996


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
# normalize data
scaler = StandardScaler();
X_scaled = scaler.fit_transform(X);
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_scaled, y, test_size=0.3, random_state=42);
# run SVC
svm_model = svm.SVC(kernel='linear', probability=False);
scores = cross_val_score(svm_model, X_train_n, y_train_n, cv=5);
print(scores);